# Simuladores de fila
## Modelagem e Avaliação de Desempenho (MAD)
## Grupo

- Ana Carolina Ferreira de Figueiredo
- Andrew Faria
- Daniel Arruda Ponte
- Paulo Yamagishi

## Imports e Constantes

In [24]:
import numpy as np
import heapq
import scipy
import pandas as pd
import plotly.express as px

CHEGADA = 0
SERVICO = 1

## Implementação




### Simulador 1 (M/M/1)

In [25]:
def simulador1(taxa_chegada, taxa_servico, servico_limite):
  clientes = 0  # Clientes na fila
  tempo_atual = 0
  numero_servicos = 0
  eventos_log = []
  rng = np.random.default_rng()

  while numero_servicos <= servico_limite:
    tprox_chegada = rng.exponential(1/taxa_chegada)
    tprox_servico = rng.exponential(1/taxa_servico)

    if clientes == 0 or tprox_chegada < tprox_servico:
      tempo_atual += tprox_chegada
      clientes += 1
      eventos_log.append((tempo_atual, CHEGADA, clientes))
    else:
      tempo_atual += tprox_servico
      numero_servicos += 1
      clientes -= 1
      eventos_log.append((tempo_atual, SERVICO, clientes))

  return eventos_log

### Simulador 2 (M/M/1)

In [26]:
def simulador2(taxa_chegada, taxa_servico, servico_limite):
   tempo_atual = 0
   clientes = 0
   numero_servicos = 0
   eventos_log = []

   rng = np.random.default_rng()

   lista_de_eventos = [(rng.exponential(1/taxa_chegada), CHEGADA)]
 
   while numero_servicos < servico_limite:  # Tempo em que a simulação ocorre
    tempo_evento, tipo_evento = heapq.heappop(lista_de_eventos) # Pego o evento de maior prioridade (próximo evento de acordo com o tempo)
    
    if tipo_evento == CHEGADA:
      tempo_atual = tempo_evento # Mudo o tempo atual para o tempo do próximo evento
      clientes += 1
      eventos_log.append((tempo_atual, CHEGADA, clientes))  
      tempo_chegada = rng.exponential(1/taxa_chegada) + tempo_atual # Calculando o tempo da próxima chegada
      heapq.heappush(lista_de_eventos, (tempo_chegada, CHEGADA))  # Coloco na fila de prioridade o próximo evento de chegada

      if clientes == 1: # Agendar o próximo serviço quando resta apenas uma pessoa na fila
        tempo_servico = rng.exponential(1/taxa_servico) + tempo_atual
        heapq.heappush(lista_de_eventos, (tempo_servico, SERVICO))
    
    elif tipo_evento == SERVICO:
      tempo_atual = tempo_evento # Mudo o tempo atual para o tempo do próximo evento
      clientes -= 1
      eventos_log.append((tempo_atual, SERVICO, clientes))
      numero_servicos += 1

      if clientes > 0: # Agendar o próximo atendimento
        tempo_servico = rng.exponential(1/taxa_servico) + tempo_atual
        heapq.heappush(lista_de_eventos, (tempo_servico, SERVICO))
   
   return eventos_log

### Simulador 1 (M/G/1 - Serviços Determinísticos)

In [27]:
def simulador1_deterministico(taxa_chegada, taxa_servico, servico_limite):
  clientes = 0  # Clientes na fila
  tempo_atual = 0
  numero_servicos = 0
  eventos_log = []
  rng = np.random.default_rng()

  while numero_servicos <= servico_limite:
    tprox_chegada = rng.exponential(1/taxa_chegada)
    tprox_servico = 1/taxa_servico

    if clientes == 0 or tprox_chegada < tprox_servico:
      tempo_atual += tprox_chegada
      clientes += 1
      eventos_log.append((tempo_atual, CHEGADA, clientes))
    else:
      tempo_atual += tprox_servico
      numero_servicos += 1
      clientes -= 1
      eventos_log.append((tempo_atual, SERVICO, clientes))

  return eventos_log

### Simulador 2 (M/G/1 - Serviços Determinísticos)

In [28]:
def simulador2_deterministico(taxa_chegada, taxa_servico, servico_limite):
   tempo_atual = 0
   clientes = 0
   numero_servicos = 0
   eventos_log = []

   rng = np.random.default_rng()

   lista_de_eventos = [(rng.exponential(1/taxa_chegada), CHEGADA)]
 
   while numero_servicos < servico_limite:  # Tempo em que a simulação ocorre
    tempo_evento, tipo_evento = heapq.heappop(lista_de_eventos) # Pego o evento de maior prioridade (próximo evento de acordo com o tempo)
    
    if tipo_evento == CHEGADA:
      tempo_atual = tempo_evento # Mudo o tempo atual para o tempo do próximo evento
      clientes += 1
      eventos_log.append((tempo_atual, CHEGADA, clientes))  
      tempo_chegada = rng.exponential(1/taxa_chegada) + tempo_atual # Calculando o tempo da próxima chegada
      heapq.heappush(lista_de_eventos, (tempo_chegada, CHEGADA))  # Coloco na fila de prioridade o próximo evento de chegada

      if clientes == 1: # Agendar o próximo serviço quando resta apenas uma pessoa na fila
        tempo_servico = 1/taxa_servico + tempo_atual
        heapq.heappush(lista_de_eventos, (tempo_servico, SERVICO))
    
    elif tipo_evento == SERVICO:
      tempo_atual = tempo_evento # Mudo o tempo atual para o tempo do próximo evento
      clientes -= 1
      eventos_log.append((tempo_atual, SERVICO, clientes))
      numero_servicos += 1

      if clientes: # Agendar o próximo atendimento
        tempo_servico = 1/taxa_servico + tempo_atual
        heapq.heappush(lista_de_eventos, (tempo_servico, SERVICO))
   
   return eventos_log

### Métricas de Interesse

In [29]:
def calcular_tempo_espera(eventos_log):
  chegadas = []
  tempo_espera = []

  for tempo_evento, tipo_evento, _ in eventos_log:
    if tipo_evento == CHEGADA:
      heapq.heappush(chegadas, tempo_evento)
    else:
      tempo_espera.append(tempo_evento - heapq.heappop(chegadas))

  return tempo_espera

In [30]:
def calcular_tempo_medio(eventos_log):
  return np.mean(calcular_tempo_espera(eventos_log)) 

In [31]:
def calcular_media_clientes(eventos_log):
  soma_clientes = 0
  
  for i in range(len(eventos_log)-1):
    soma_clientes += (eventos_log[i+1][0] - eventos_log[i][0]) * eventos_log[i][2]

  return soma_clientes / eventos_log[-1][0]

### Testes

In [32]:
# Teste Simulador 1
lista = simulador1(1, 2, 100000)
print(calcular_tempo_medio(lista))
print(calcular_media_clientes(lista))

1.0065229659390336
1.0100286526486053


In [33]:
# Teste Simulador 2
lista = simulador2(1, 2, 1000)
print(calcular_tempo_medio(lista))
print(calcular_media_clientes(lista))

0.8293366211771199
0.8124735371240173


In [34]:
# Teste Simulador 2 (Serviços determinísticos)
lista = simulador2_deterministico(1, 2, 1000)
print(calcular_tempo_medio(lista))
print(calcular_media_clientes(lista))

0.7302640757900613
0.7133788065872895


### Diversas Simulações

In [35]:
def n_simulacoes(n, taxa_chegada, taxa_servico, servico_limite, simulador):
  tempos_medios = []
  medias_clientes = []

  for i in range(n):
    lista_eventos = simulador(taxa_chegada, taxa_servico, servico_limite)

    tempos_medios.append(calcular_tempo_medio(lista_eventos))
    medias_clientes.append(calcular_media_clientes(lista_eventos))

  return tempos_medios, medias_clientes

### Intervalo de Confiança

In [36]:
def intervalo_confianca(lista):
  desvio_padrao = np.std(lista)
  conf_int = scipy.stats.norm.interval(0.95, loc= np.mean(lista), scale=desvio_padrao)

  return conf_int

In [37]:
t_medio, m_clientes = n_simulacoes(300, 1, 2, 1000, simulador1)

In [38]:
print(intervalo_confianca(t_medio))
print(intervalo_confianca(m_clientes))

(0.8097865696694198, 1.1703329886585863)
(0.7778954970033235, 1.2005352049619444)


## Gráficos e métricas de interesse

In [39]:
def plotar_histograma(data, label):
  x = [x for x in range(len(data))]

  d = { 'x': x, label: data }
  d = dict(sorted(d.items(), key=lambda x : x[1]))

  df = pd.DataFrame(data=d)
  fig = px.histogram(df, x=label).show()

### Cenário 1
$λ=1$ e $μ=2$

#### Simulador 1

In [40]:
tempo_espera, clientes_sistema = n_simulacoes(300, 1, 2, 1000, simulador1)

##### Tempo de espera



In [41]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (0.8177237645936763, 1.1832538151093512)
Média: 1.0004887898515138



##### Clientes no sistema

In [42]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (0.7915334597634868, 1.2173778163484004)
Média: 1.0044556380559435



#### Simulador 2

In [43]:
tempo_espera, clientes_sistema = n_simulacoes(300, 1, 2, 1000, simulador2)

##### Tempo de espera

In [44]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (0.8278039774883263, 1.1752668106718025)
Média: 1.0015353940800644



##### Clientes no sistema

In [45]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (0.7990582697171229, 1.2091731406594985)
Média: 1.0041157051883107



### Cenário 2

$λ=2$ e $μ=4$

#### Simulador 1

In [46]:
tempo_espera, clientes_sistema = n_simulacoes(300, 2, 4, 1000, simulador1)

##### Tempo de espera



In [47]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (0.4052268021904245, 0.5868061996693846)
Média: 0.49601650092990457



##### Clientes no sistema

In [48]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (0.7898314874636875, 1.2024440253955295)
Média: 0.9961377564296086



#### Simulador 2

In [49]:
tempo_espera, clientes_sistema = n_simulacoes(300, 2, 4, 1000, simulador2)

##### Tempo de espera



In [50]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (0.4059301157626023, 0.5992581305297724)
Média: 0.5025941231461873



##### Clientes no sistema

In [51]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (0.7858866041613173, 1.2318102970711184)
Média: 1.0088484506162179



### Cenário 3

$λ=1$ e $μ=4$ 

#### Simulador 1

In [52]:
tempo_espera, clientes_sistema = n_simulacoes(300, 1, 4, 1000, simulador1)

##### Tempo de espera



In [53]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (0.2996810279162096, 0.3633830605492348)
Média: 0.3315320442327222



##### Clientes no sistema

In [54]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (0.2916935647590087, 0.3724138434342457)
Média: 0.3320537040966272



#### Simulador 2

In [55]:
tempo_espera, clientes_sistema = n_simulacoes(300, 1, 4, 1000, simulador2)

##### Tempo de espera



In [56]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (0.3011601128136674, 0.3672015466252476)
Média: 0.3341808297194575



##### Clientes no sistema

In [57]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (0.29100336624811723, 0.37781417429046543)
Média: 0.33440877026929133



### Cenário 4

$λ=1$ e $μ=2$ (Serviço com tempo determinístico)

#### Simulador 1 (Serviços Determinístico)

In [58]:
tempo_espera, clientes_sistema = n_simulacoes(300, 1, 2, 1000, simulador1_deterministico)

#####Tempo de espera

In [59]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (1.3057532947932948, 2.326602529418223)
Média: 1.8161779121057589



#####Clientes no Sistema

In [60]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (1.2335836156004867, 2.409012742184313)
Média: 1.8212981788923996



#### Simulador 2 (Serviços Determinístico)

In [61]:
tempo_espera, clientes_sistema = n_simulacoes(300, 1, 2, 1000, simulador2_deterministico)

##### Tempo de espera



In [62]:
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

plotar_histograma(tempo_espera, 'Tempo de espera')

Intervalo de confiança: (0.6974434767325599, 0.8032934765084836)
Média: 0.7503684766205218



##### Clientes no sistema

In [63]:
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

plotar_histograma(clientes_sistema, 'Clientes no sistema')

Intervalo de confiança: (0.6634894231405919, 0.8403685515518251)
Média: 0.7519289873462085

